In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_teddynote import logging
logging.langsmith("CH21-AGENT")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH21-AGENT


# Tools 

- 도구(Tool) : Agent, Chain 또는 LLM이 외부와 상호작용 하기 위한 인터페이스 
- LangChain에서 기본적으로 제공하는 도구, 또는 Custom Tool 구성하여 적용 
- LangChain 도구 리스트 : https://python.langchain.com/v0.1/docs/integrations/tools/


## Built-in Tools 
- LangChain에서 제공하는 사전 정의된 took & toolkit 
    - tool : 단일 도구 
    - toolkit : 여러 도구를 묶어서 하나의 도구로 사용할 수 있음 
    - https://python.langchain.com/docs/integrations/tools/

### Python REPL Tools 
- 파이썬 코드를 REPL 환경에서 실행하기 위한 클래스 제공 
    - REPL : Read-Eval-Print Loop, 파이썬을 바로 실행해 볼 수 있는 대화형 환경 
    - https://python.langchain.com/docs/integrations/tools/python/


In [3]:
from langchain_experimental.tools import PythonREPLTool

In [4]:
python_tool = PythonREPLTool()

print(python_tool.invoke(
    "print (100 + 500)"
))

Python REPL can execute arbitrary code. Use with caution.


600



#### LLM 적용 

기본 흐름 
1. LLM 모델에게 특정 작업을 수행하는 파이썬 코드 작성 요청 
2. 작성된 코드를 실행하여 결과 획득 
3. 결과 출력

In [5]:
from langchain_openai import OpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

In [10]:
def print_and_execute(code, debug=True):
    if debug:
        print("CODE: ")
        print(code)
    
    return python_tool.invoke(code)

In [11]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are Raymond Hetting, an expert python programmer, well versed in meta-programming and elegant, concise and short but well documented code. You follow the PEP8 style guide. "
            "Return only the code, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the code.",
        ),
        ("human", "{input}"),
    ]
)

llm = OpenAI(temperature=0)

chain = prompt | llm | StrOutputParser() | RunnableLambda(print_and_execute)

In [12]:
print(chain.invoke("로또 번호 생성기를 출력하는 코드를 작성하세요."))

CODE: 


import random

# Generate 6 random numbers between 1 and 45
lotto_numbers = random.sample(range(1, 46), 6)

# Sort the numbers in ascending order
lotto_numbers.sort()

# Print the numbers
print(lotto_numbers)
[1, 4, 23, 26, 32, 34]



### 검색API 도구

- Tavily API : 검색 기능을 구현해 놓은 도구 
- [Ref] https://python.langchain.com/docs/integrations/tools/tavily_search/

- 사전준비 : 
    - API 키 발급 : https://app.tavily.com/
    - 발급받은 키를 `.env` 파일에 반영 (TAVILY_API_KEY=tv...)

- 주요 클래스 : 
    - `TavilyAnswer`
    - `TavilySerachResults`
        - API 통해 검색하고 JSON 형식을 결과를 반환 
        - 주요 매개 변수 
            - `max_results` (int): 반환할 최대 검색 결과 수 (기본값: 5)
            - `search_depth` (str): 검색 깊이 ("basic" 또는 "advanced")
            - `include_domains` (List[str]): 검색 결과에 포함할 도메인 목록
            - `exclude_domains` (List[str]): 검색 결과에서 제외할 도메인 목록
            - `include_answer` (bool): 원본 쿼리에 대한 짧은 답변 포함 여부
            - `include_raw_content` (bool): 각 사이트의 정제된 HTML 콘텐츠 포함 여부
            - `include_images` (bool): 쿼리 관련 이미지 목록 포함 여부

In [13]:
from langchain_core.pydantic_v1 import BaseModel
from langchain_community.tools.tavily_search import TavilySearchResults

In [14]:
# 도구 생성 
tool = TavilySearchResults(
    max_results=6,
    include_answer=True,
    include_raw_content=True,
    # include_images=True,
    # search_depth="advanced" # or "basic"
    include_domains=["github.io","wikidocs.net"],
    # exclude_domains=[]
)

In [16]:
tool.invoke({
    "query":"LangChain Tool에 대해서 알려주세요"
})

[{'url': 'https://langchain-ai.github.io/langgraph/tutorials/introduction/',
  'content': '[](https://langchain-ai.github.io/langgraph/tutorials/introduction/#__codelineno-9-1)Assistant: LangGraph is a library designed to help build stateful multi-agent applications using language models. It provides tools for creating workflows and state machines to coordinate multiple AI agents or language model interactions. LangGraph is built on top of LangChain, leveraging its components while adding graph-based coordination capabilities. It\'s particularly useful for developing more complex, [...] [](https://langchain-ai.github.io/langgraph/tutorials/introduction/#__codelineno-21-9)   LangGraph is developed by LangChain, a company known for its tools and frameworks in the AI and LLM space.\n[](https://langchain-ai.github.io/langgraph/tutorials/introduction/#__codelineno-21-10)\n[](https://langchain-ai.github.io/langgraph/tutorials/introduction/#__codelineno-21-11)3. Key Features: [...] [](https:/

## Custom Tools 

- LangChain에서 제공하는 빌트인 도구와 별도로, 사용자가 직접 도구를 정의하교 사용 
- 이를 위해 `langchain.tools` 모듈에서 제공하는 `tool` decorator를 사용하여 파이썬 함수를 도구로 변환 

### @tool decorator 
- 파이썬 함수를 도구로 변환 하는 기능 
- 사용 방법 
    - 함수 위에 `@tool` 데코레이터 적용 
    - 필요에 따라 데코레이터 매개변수 설정 

In [17]:
from langchain.tools import tool

In [18]:
# 데코레이터 추가하여 함수 -> 도구로 변환 
# LLM이 함수를 호출할 때 함수의 역할을 확인할 수 있도록 주석을 반드시 추가 (in English)

@tool
def add_numbers(a: int, b: int) -> int:
    """ Add two numbers
    """
    return a + b

@tool
def mult_numbers(a: int, b: int) -> int:
    """ Multiply two numbers
    """
    return a * b

In [19]:
add_numbers.invoke({'a': 10, 'b':5})

15

In [20]:
mult_numbers.invoke({'a': 10, 'b':5})

50

--------
** End of Documents **